In [4]:
import yfinance as yf
# yfinance pre installed

WFC_DF = yf.download('WFC', start = '2018-4-29',end='2022-04-29')
WFC_ticker = yf.Ticker('WFC')

BAC_DF = yf.download('BAC', start = '2018-4-29',end='2022-04-29')
BAC_ticker = yf.Ticker('BAC')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [9]:
# Calculating daily returns

def Daily_R(Open, Close):
    Daily_return = Open - Close
    return round(Daily_return,5)

WFC_DF['DR'] = WFC_DF[
    ['Open','Close']].apply(
    lambda df: Daily_R(
        df['Open'],df['Close']), axis=1)

BAC_DF['DR'] = BAC_DF[
    ['Open','Close']].apply(
    lambda df: Daily_R(
        df['Open'],df['Close']), axis=1)

# axis 1 means rows.
#.apply applies the function across the selected dataframe columns.
#lambda is a small one time function, here is is required.

In [10]:
WFC_DF

,Open,High,Low,Close,Adj Close,Volume,DR
Date,,,,,,,
2018-04-30,52.560001,53.049999,51.959999,51.959999,44.958458,17050500,0.60
2018-05-01,51.759998,52.599998,51.599998,52.560001,45.477612,13903800,-0.80
2018-05-02,52.380001,52.869999,52.060001,52.189999,45.157471,16397200,0.19
2018-05-03,51.619999,51.790001,50.810001,51.540001,44.930809,19216100,0.08
2018-05-04,51.169998,52.900002,51.070000,52.410000,45.689243,16660500,-1.24
...,...,...,...,...,...,...,...
2022-04-22,47.689999,47.919998,46.259998,46.340000,45.191826,21676800,1.35
2022-04-25,45.759998,45.970001,44.529999,45.830002,44.694466,30045500,-0.07
2022-04-26,45.160000,46.000000,44.560001,44.580002,43.475433,21527400,0.58


In [11]:
BAC_DF

,Open,High,Low,Close,Adj Close,Volume,DR
Date,,,,,,,
2018-04-30,30.270000,30.430000,29.920000,29.920000,26.750227,55153900,0.35
2018-05-01,29.920000,29.959999,29.620001,29.950001,26.777050,52871700,-0.03
2018-05-02,29.950001,30.070000,29.559999,29.580000,26.446245,68234600,0.37
2018-05-03,29.510000,29.520000,28.430000,29.200001,26.106504,110672700,0.31
2018-05-04,28.990000,29.420000,28.820000,29.299999,26.195910,58281900,-0.31
...,...,...,...,...,...,...,...
2022-04-22,38.810001,38.849998,37.500000,37.560001,36.647125,50409800,1.25
2022-04-25,36.889999,37.400002,35.930000,37.290001,36.383690,64301100,-0.40
2022-04-26,36.880001,37.540001,36.419998,36.450001,35.564102,44809900,0.43


In [ ]:
# Parse dates
# Set index to dates
# 3 day average
# rolling standard deviation 
# Daily & Annual Sharpie Ratio
# Daily & Annual Sortino Ratio